<a href="https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This code is based on a tutorial made publicly available by google colab.

# This Colab demonstates using a free Colab Cloud TPU to fine-tune sentence and sentence-pair classification tasks built on top of pretrained BERT models and run predictions on tuned model.
# The colab demonsrates loading pretrained BERT models from both TF Hub and checkpoints.

# https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb

In [ ]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import pandas as pd 

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.120.20.194:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 4998319205859215909),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9386882932018178009),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7559679506075455314),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2373243628037609622),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1536991947415538577),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 16063516450798020116),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 11259345396115279846),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 9203572266884672480),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1330224615364

### Prepare and import BERT modules
​
With your environment configured, you can now prepare and import the BERT modules. The following step clones the source code from GitHub and import the modules from the source. Alternatively, you can install BERT using pip (!pip install bert-tensorflow).

In [ ]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

### Prepare for training

This next section of code performs the following tasks:

*  Specify task and download training data.
*  Specify BERT pretrained model
*  Specify GS bucket, create output directory for model checkpoints and eval results.




In [ ]:
TASK = 'MRPC' #@param {type:"string"}
assert TASK in ('MRPC', 'CoLA'), 'Only (MRPC, CoLA) are demonstrated here.'

# Download glue data.
! test -d download_glue_repo || git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git download_glue_repo
!python download_glue_repo/download_glue_data.py --data_dir='glue_data' --tasks=$TASK

TASK_DATA_DIR = 'glue_data/' + TASK
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))

BUCKET = 'bert-test-clyde2' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Cheat the model to use our train set instead of the MRPC one
print('***** Copying bucket data to the task directory *****')
tf.gfile.Copy('gs://'+BUCKET+'/train.tsv',TASK_DATA_DIR+'/train.tsv',True)
tf.gfile.Copy('gs://'+BUCKET+'/dev.tsv',TASK_DATA_DIR+'/dev.tsv',True)
tf.gfile.Copy('gs://'+BUCKET+'/test.tsv',TASK_DATA_DIR+'/test.tsv',True)

!ls $TASK_DATA_DIR

BERT_MODEL = 'multi_cased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!
***** Task data directory: glue_data/MRPC *****
***** Model output directory: gs://bert-test-clyde2/bert-tfhub/models/MRPC *****
***** Copying bucket data to the task directory *****
dev_ids.tsv  msr_paraphrase_test.txt   test.tsv
dev.tsv      msr_paraphrase_train.txt  train.tsv


Now let's load tokenizer module from TF Hub and play with it.

In [ ]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
print("English: \t", tokenizer.tokenize("This here's an example of using the BERT tokenizer"))
print("Dutch: \t\t", tokenizer.tokenize("Wij zijn voor projecten in Alkmaar e.o. opzek naar een ervaren interieurvergzorgster"))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


English: 	 ['This', 'here', "'", 's', 'an', 'example', 'of', 'using', 'the', 'BE', '##RT', 'tok', '##eni', '##zer']
Dutch: 		 ['Wi', '##j', 'zijn', 'voor', 'projecte', '##n', 'in', 'Al', '##km', '##aar', 'e', '.', 'o', '.', 'op', '##zek', 'naar', 'een', 'er', '##vare', '##n', 'inter', '##ieur', '##ver', '##g', '##zor', '##gste', '##r']


**Also** we initilize our hyperprams, prepare the training data and initialize TPU config.

In [ ]:
TRAIN_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 8
# PREDICT_BATCH_SIZE = 1032
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 500

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()

# Compute number of train and warmup steps from batch size
# Task Data Dir can be changed by the data dir needed
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 2000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

print("Length of the training examples:", len(train_examples))
print("Amount of training steps:\t", num_train_steps)
print("Amount of warmup steps:\t\t", num_train_steps)

train_examples = processor.get_train_examples(TASK_DATA_DIR)
test_examples = processor.get_test_examples(TASK_DATA_DIR)
dev_examples = processor.get_dev_examples(TASK_DATA_DIR)

print("Length of the training examples:", len(train_examples))
print("Length of the test examples:", len(test_examples))
print("Length of the dev examples:", len(dev_examples))

Length of the training examples: 2240
Amount of training steps:	 105
Amount of warmup steps:		 105
Length of the training examples: 2240
Length of the test examples: 342930
Length of the dev examples: 760


# Fine-tune and Run Predictions on a pretrained BERT Model from TF Hub

This section demonstrates fine-tuning from a pre-trained BERT TF Hub module and running predictions.


In [ ]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR


model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE

  
)

INFO:tensorflow:Using config: {'_model_dir': 'gs://bert-test-clyde2/bert-tfhub/models/MRPC', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.120.20.194:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f37d41ce710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.120.20.194:8470', '_evaluation_master': 'grpc://10.120.20.194:8470', '_is_chief': True, '_

INFO:tensorflow:Using config: {'_model_dir': 'gs://bert-test-clyde2/bert-tfhub/models/MRPC', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.120.20.194:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f37d41ce710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.120.20.194:8470', '_evaluation_master': 'grpc://10.120.20.194:8470', '_is_chief': True, '_

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


At this point, you can now fine-tune the model, evaluate it, and run predictions on it.

In [ ]:
# Train the model
def model_train(estimator):
  print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
  # Sequences length is set to 128 tokens.
  train_features = run_classifier.convert_examples_to_features(train_examples,
                                                               label_list, 
                                                               MAX_SEQ_LENGTH,
                                                               tokenizer)
  
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  tf.logging.set_verbosity(tf.logging.INFO)

  train_input_fn = run_classifier.input_fn_builder(features=train_features,
                                                   seq_length=MAX_SEQ_LENGTH,
                                                   is_training=True,
                                                   drop_remainder=True)
  
  test_var = estimator.train(input_fn=train_input_fn,
                  max_steps=num_train_steps)
  
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))
  return test_var

In [ ]:
test_var = model_train(estimator_from_tfhub)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...



INFO:tensorflow:Writing example 0 of 2240


INFO:tensorflow:Writing example 0 of 2240


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Loss for final step: 0.8900251.


INFO:tensorflow:Loss for final step: 0.8900251.


INFO:tensorflow:training_loop marked as finished


INFO:tensorflow:training_loop marked as finished


***** Finished training at 2020-05-12 12:03:56.103701 *****


In [ ]:
print(tf.summary)

<module 'tensorflow._api.v1.summary' from '/tensorflow-1.15.2/python3.6/tensorflow_core/_api/v1/summary/__init__.py'>


In [ ]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(eval_examples, 
                                                              label_list, 
                                                              MAX_SEQ_LENGTH, 
                                                              tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(features=eval_features,
                                                  seq_length=MAX_SEQ_LENGTH,
                                                  is_training=False,
                                                  drop_remainder=True)
  
  result = estimator.evaluate(input_fn=eval_input_fn,
                              steps=eval_steps)
  
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))


In [ ]:
model_eval(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 760


INFO:tensorflow:Writing example 0 of 760


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: dev-1


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 105: gs://bert-test-clyde2/bert-tfhub/models/MRPC/model.ckpt-105


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 105: gs://bert-test-clyde2/bert-tfhub/models/MRPC/model.ckpt-105


INFO:tensorflow:evaluation_loop marked as finished


INFO:tensorflow:evaluation_loop marked as finished


***** Finished evaluation at 2020-05-12 12:04:45.796062 *****
***** Eval results *****
  eval_accuracy = 0.8513158
  eval_loss = 0.6204963
  global_step = 105
  loss = 0.6997512


For training the model only, we can stop at this point running the rest of the notebook


In [ ]:
import pandas as pd 

def model_predict(estimator):
  # Make predictions on all of eval examples
  prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:]
  input_features = run_classifier.convert_examples_to_features(prediction_examples,
                                                               label_list,
                                                               MAX_SEQ_LENGTH,
                                                               tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features,
                                                     seq_length=MAX_SEQ_LENGTH,
                                                     is_training=False,
                                                     drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)

  pred_list = []
  
  for example, prediction in zip(prediction_examples, predictions):
#     print('text_a: %s\ntext_b: %s\nlabel:%s\nprediction:%s\n' % (example.text_a, example.text_b, str(example.label), prediction['probabilities']))
    if (prediction['probabilities'][0] > prediction['probabilities'][1]):
      predicted = 0
    else:
      predicted = 1
   # print(dir(example))
   # print(prediction)
    pred_list.append({'text_a': example.text_a,
                      'orig_label': example.label,
                      'prediction_probabilities': prediction['probabilities'],
                      'predicted':predicted})

  return pred_list

In [ ]:
prediction_list = model_predict(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 760


INFO:tensorflow:Writing example 0 of 760


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: dev-1


...

INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.


INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:prediction_loop marked as finished


INFO:tensorflow:prediction_loop marked as finished


In [ ]:
df = pd.DataFrame(data=prediction_list)
# Example of first 5 rows
print(df[["orig_label", "prediction_probabilities", "predicted"]].head(20))
print(len(df))


# df.to_csv('predictions.csv')

# #Write to bucket 
# #tf.gfile.Copy(oldpath, newpath, overwrite=False)
# tf.gfile.Copy('/content/predictions.csv', 'gs://'+BUCKET+'/bert-tfhub/my_data/multi/predictions.csv', True)

   orig_label    prediction_probabilities  predicted
0           1     [0.02079578, 0.9792043]          1
1           1   [0.9973545, 0.0026454814]          0
2           1     [0.016118977, 0.983881]          1
3           1    [0.01809464, 0.98190534]          1
4           0  [0.99868006, 0.0013199179]          0
5           0    [0.9988416, 0.001158443]          0
6           0   [0.99877626, 0.001223737]          0
7           1    [0.01500777, 0.98499227]          1
8           0  [0.99863344, 0.0013665445]          0
9           0   [0.9986513, 0.0013487114]          0
10          1    [0.011427351, 0.9885726]          1
11          1   [0.016679933, 0.98332006]          1
12          1    [0.031392198, 0.9686079]          1
13          1    [0.02808014, 0.97191983]          1
14          1    [0.9757932, 0.024206867]          0
15          0   [0.012535495, 0.98746455]          1
16          1      [0.0164536, 0.9835463]          1
17          1     [0.05026054, 0.9497395]     

In [ ]:
eval_predicted = df[['predicted']].values.astype(int)
eval_predicted = eval_predicted.ravel()
test_target = df[['orig_label']].values.astype(int)
test_target = test_target.ravel()

print(test_target[0:10])
print(type(test_target))


[1 1 1 1 0 0 0 1 0 0]
<class 'numpy.ndarray'>


In [ ]:
import sklearn.metrics as sk

# evaluation_score = classifier.score(X_test, test_target)
# print("Evaluation Accuracy:", evaluation_score)

# eval_predicted = classifier.predict(X_test)

precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

Eval Precision:  0.8909574468085106
Eval recall:  0.8230958230958231
Eval roc_auc:  0.8534742571569767
Eval f1:  0.8556832694763729
